In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from glob import glob
import random

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
W, H, D = 224, 224, 3

C:\Users\mmill\Anaconda3\envs\tensorflow25\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mmill\Anaconda3\envs\tensorflow25\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\mmill\Anaconda3\envs\tensorflow25\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
encodings = []
path = r'F:\hac\custom_dataset'
ids = os.listdir(path)
for index, i in enumerate(ids[:]):
    encodings.append([])
    names = os.listdir(os.path.join(path, i))
    for k, name in tqdm(enumerate(names), desc=f'{index + 1}/{len(ids)}', total=len(names)):
        img = image.load_img(os.path.join(path, i, name), target_size=(W,H,D))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis=0)
        img_tensor /= 255.
        encodings[index].append(model.predict(img_tensor))

102/102: 100%|███████████████████████████████████████████████████████████████████████| 464/464 [00:19<00:00, 23.90it/s]


In [6]:
import pandas as pd

names = []

for i in ids:
    names.extend([os.path.join(i, name) for name in os.listdir(os.path.join(path, i))])


In [9]:
enc = []
for arr in encodings:
    enc.extend(arr)

In [11]:
df = pd.DataFrame()
df['ids'] = names
df['encodings'] = enc
df

,ids,encodings
0,1\0.jpg,"[[0.039624333, 0.16381598, -0.065188415, -0.08..."
1,1\1.jpg,"[[0.042282194, 0.05630434, -0.02977801, -0.020..."
2,1\10.jpg,"[[0.051137805, 0.15925086, -0.07144881, -0.074..."
3,1\100.jpg,"[[0.015196406, 0.018183826, 0.011113236, -0.00..."
4,1\101.jpg,"[[0.021339476, 0.019626467, 0.010315144, -0.00..."
...,...,...
33638,99\95.jpg,"[[0.002795008, 0.015622405, 0.0042289738, 0.01..."
33639,99\96.jpg,"[[0.005694085, 0.013091637, -0.0011384544, 0.0..."
33640,99\97.jpg,"[[0.002534815, 0.016474051, 0.006219143, 0.018..."
33641,99\98.jpg,"[[0.0016266451, 0.021133518, 0.0015703681, 0.0..."


In [13]:
df.to_csv('whale_encodings.csv', index=False)

In [14]:
encodings = list(df['encodings'])
len(encodings)

33643

In [15]:
for i in tqdm(range(len(encodings)), total=len(encodings)):
    euclidean_distance([encodings[0], encodings[i]])

100%|██████████████████████████████████████████████████████████████████████████| 33643/33643 [00:08<00:00, 3956.47it/s]


In [2]:
num_classes = 102
def create_pairs(data, digit_indices):
    pairs = []
    labels = []
    count = 0
# Получаем минимальное количество одинаковых чисел в базе
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            # Создаем верную пару
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[data[z1], data[z2]]]
            # Создаем не верную пару
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            # Записываем в список
            pairs += [[data[z1], data[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [3]:
num_classes = 102
path = r'F:\hac\custom_dataset'
ids = os.listdir(path)
ids = [int(i) for i in ids]
ids = sorted(ids)
x_path = []
y = []
l_n = [len(os.listdir(os.path.join(path, name))) for name in os.listdir(path)]
for i in tqdm(ids):
    names = os.listdir(os.path.join(path, str(i)))
    names2 = [int(name.split('.')[0]) for name in names]
    names = [str(i) + '.jpg' for i in sorted(names2)]
#     if len(names) < 100:
#         step = 1
#     elif 100 < len(names) < 400:
#         step = 5
#     else:
#         step = 10
    for j, name in enumerate(names[:]):
        for k in range(5):
            inc = random.randint(1, l_n[i - 1])
            index = (int(i) - 1 + inc) % len(names)
            x_path.append([os.path.join(path, str(i), name), os.path.join(path, str(i), names[index])])  # 1   
            y.append(1.)
            inc = random.randrange(1, num_classes)
            dn = (int(i) - 1 + inc) % num_classes
            
            inc = random.randrange(1, l_n[int(dn)])
            index = (int(dn) + inc) % l_n[int(dn)]
            x_path.append([os.path.join(path, str(i), name), os.path.join(path, str(dn), name)]) # 0
            y.append(0.)
#             if k == 1:
#                 break
#         break
#     break

100%|██████████| 102/102 [00:03<00:00, 32.26it/s]


In [7]:
with open('x_path.txt', 'w') as file:
    for line in x_path:
        file.write(str(line[0]) + ' ' + str(line[1]) +'\n')

with open('y.txt', 'w') as file:
    for line in y:
        file.write(str(line) +'\n')

In [18]:
x_path = []
y = []
with open('x_path.txt', 'r') as file:
    for line in file:
        x_path.append(line[:-1].split(' '))
        
with open('y.txt', 'r') as file:
    for line in file:
        y.append(float(line))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_path, y, test_size=0.25, random_state=42)
# x_train, x_test, y_train, y_test

In [24]:
from vit_keras import vit, utils

image_size = W
classes = utils.get_imagenet_classes()
vit_model = vit.vit_b16(
    image_size=image_size,
    activation='sigmoid',
    pretrained=True,
    include_top=False,
    pretrained_top=False
)
classToken = vit_model.layers[3]
# vit_model = None

In [25]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *


W, H, D = 224, 224, 3
def create_base_network(input_shape):
    '''Базовая сеть для совместного использования (eq. to feature extraction).
    '''
    model = Sequential()
#     resnet_model = Sequential()

#     pretrained_model= tf.keras.applications.ResNet50(weights="imagenet",
#                                                      input_shape=(W, H, D),
#                                                      include_top=False,
#                                                      pooling='avg'
#                                                     )
    pretrained_model = vit_model
#     for layer in pretrained_model.layers:
#             layer.trainable=1
#     trainable = False
#     for layer in model.layers:
#         if layer.name == "conv5_block1_out":
#             trainable = True
#         layer.trainable = trainable

    model.add(pretrained_model)
    
    # Слой свертки
#     model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
#     model.add(MaxPooling2D(2))
    # Слой свертки
#     model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
#     model.add(MaxPooling2D(2))
    
#     model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
#     model.add(MaxPooling2D(2))
    
#     model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
#     model.add(MaxPooling2D(2))
    
#     model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
#     model.add(MaxPooling2D(2))
    
#     model.add(Conv2D(32, kernel_size=3, activation='relu'))
#     model.add(MaxPooling2D(2))

    model.add(Flatten())
    # Полносвязный слой
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256))
    
    

    return model

In [26]:
input_shape = (W,H,D)
base_network = create_base_network(input_shape)
input_a = Input(shape=input_shape, name='input_1')
input_b = Input(shape=input_shape, name='input_2')
# Получаем вектора признаков
processed_a = base_network(input_a)
processed_b = base_network(input_b)

In [27]:
from tensorflow.keras import backend as K

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

In [28]:
# def contrastive_loss(y_true, y_pred):
#     '''Контрастная потеря от Hassell-et-al.'06
#     http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
#     '''
#     margin = 1
#     square_pred = K.square(y_pred)
#     margin_square = K.square(K.maximum(margin - y_pred, 0))
#     return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss


def accuracy(y_true, y_pred):
    '''Вычислить точность классификации с фиксированным порогом по расстояниям.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [29]:
model = Model([input_a, input_b], distance)
# Оптимизатор будет RMSprop
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
        
# Собираем модель
model.compile(loss=contrastive_loss, optimizer=opt, metrics=[accuracy])

model.summary()

tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=1)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 256)          86392576    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1)            0           sequential_2[0][0]         

InvocationException: Program terminated with status: 1. stderr follows: Error: C:\Users\mmill\AppData\Local\Temp\tmpv7lomg06: syntax error in line 13 near '-'


In [ ]:
tf.keras.utils.plot_model(model.layers[2], show_shapes=True, expand_nested=0)

In [30]:
def gen(x, y, batch_size=2):
    def get_out(inp):
        if inp[0].split('\\')[-2] == inp[1].split('\\')[-2]:
            return 1.
        else: 
            return 0.
        
    count = 0
#     print(x[0])
    random.shuffle(x)
#     print(x[0])
    for k in range(len(x)//batch_size):
#         if 1:
        try:
            batch_paths = x[k*batch_size:(k+1)*batch_size]
            batch_input  = []
            batch_output = [] 
            input1 = []
            input2 = []
            for input_path in batch_paths:
                img = image.load_img(input_path[0],
                            target_size=(W, H, D))

                img_tensor1 = image.img_to_array(img)
                img_tensor1 /= 255.

                img = image.load_img(input_path[1], 
                            target_size=(W, H, D))

                img_tensor2 = image.img_to_array(img)
                img_tensor2 /= 255.

                output = get_out(input_path)
    #             output = y[count]
                count += 1

                input1 += [img_tensor1]
                input2 += [img_tensor2]
#                 batch_input += [[img_tensor1, img_tensor2]]
                batch_output += [output]
    
            batch_input += [input1, input2]
            batch_x = np.array( batch_input )
    #         batch_y = np.array( batch_output )
            batch_y = np.array(batch_output)
        except Exception as exc:
#             print(exc)
            continue
#         print(input_path, batch_y)
        yield ([np.array(batch_x[0]), np.array(batch_x[1])], batch_y )

In [38]:
model.fit(gen(x_train, y_train, batch_size=2),
            steps_per_epoch=200,
            epochs=1000,
            validation_data=gen(x_test, y_test, batch_size=2),
            validation_steps=100,
             )


Epoch 1/1000
200/200 [==============================] - 142s 709ms/step - loss: 277.6615 - accuracy: 0.3275 - val_loss: 1061.7429 - val_accuracy: 0.3650
Epoch 2/1000
 76/200 [==========>...................] - ETA: 1:17 - loss: 257.4707 - accuracy: 0.3750

KeyboardInterrupt: 

In [19]:
model.evaluate(gen(x_test, y_test, batch_size=8), steps=100)

100/100 [==============================] - 20s 202ms/step - loss: 42.5255 - accuracy: 0.5462


[42.5255126953125, 0.5462499856948853]

In [20]:
model.save('best_model.h5')

C:\Users\mmill\Anaconda3\envs\tensorflow25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [41]:
img1 = image.load_img(r'F:\hac\custom_dataset\1\1.jpg', target_size=(W,H,D))
img_tensor = image.img_to_array(img1)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.
img1 = img_tensor

img2 = image.load_img(r'F:\hac\custom_dataset\2\99.jpg', target_size=(W,H,D))
img_tensor = image.img_to_array(img2)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.
img2 = img_tensor
model.predict([img1, img2])

array([[41.57617]], dtype=float32)